In [1]:
import netaddr
import ipaddress
import maxminddb
import pandas as pd
from tqdm import tqdm
from io import StringIO
from netaddr import IPSet
from mmdb_writer import MMDBWriter

# Get Data 
## https://lite.ip2location.com/iran-(islamic-republic-of)-ip-address-ranges?lang=en_US
## login to the website or use this JQuery to fetch data from the html

In [2]:
'''
$('#ip-address tr').each(function() {
  var firstTdValue = $(this).find('td:nth-child(1)').text();
  var secondTdValue = $(this).find('td:nth-child(2)').text();

  commaSeparatedValues += firstTdValue + ', ' + secondTdValue + ', ';
});
''';

In [3]:
with open("./data/jQueryResult.txt", "r") as file:
    text = file.read()

In [4]:
start = text.split(", ")[0::2]
end = text.split(", ")[1::2]
df = pd.DataFrame({"start": start, "end": end})

In [5]:
cidrs = []
for index, row in df.iterrows():    
    for cidr in netaddr.iprange_to_cidrs(row['start'], row['end']):
        cidrs.append(cidr)    

In [6]:
cidrs = pd.Series(cidrs).unique()
# only use version 4 cidrs
cidrs = [cidr for cidr in cidrs if ipaddress.ip_network(cidr).version == 4]

In [7]:
def remove_not_superne(cidrs_list):    
    remove_list = []
    # calc network for each cirdr, catch for speed up
    cidr2network = {}
    for cidr in cidrs_list:
        cidr2network[cidr] = ipaddress.ip_network(cidr)
    
    for i in tqdm(range(0, len(cidrs_list))):
        cird_i = cidrs_list[i]    
        if(cird_i in remove_list):
            continue
        network1 = cidr2network[cird_i]
        for j in range(i+1, len(cidrs_list)):        
            cird_j = cidrs_list[j]        
            network2 = cidr2network[cird_j]
            if network1.supernet_of(network2):                
                remove_list.append(cird_j)            
            elif network2.supernet_of(network1):                
                remove_list.append(cird_i)  
                break        
    return remove_list 

rm = remove_not_superne(cidrs)

100%|██████████████████████████████████████| 1664/1664 [00:06<00:00, 268.99it/s]


In [8]:
print(len(cidrs))
print(len(rm))
cidrs = set(cidrs) - set(rm)
print(len(cidrs))

1664
0
1664


# MMDB Writer

In [11]:
writer = MMDBWriter()
for cidr in tqdm(cidrs):    
    writer.insert_network(IPSet([cidr]), {'country': 'IR', 'isp': "NA"})    
writer.to_db_file('geoipIR.mmdb')

100%|████████████████████████████████████| 1664/1664 [00:00<00:00, 29018.48it/s]


#  Test dataste

In [12]:
m = maxminddb.open_database('geoipIR.mmdb')
r = m.get('79.175.191.78')
r

{'country': 'IR', 'isp': 'NA'}

# Convert MMDB to DAT format